# Import Brain Inverders Library

In [1]:
from braininvaders2015a.dataset import BrainInvaders2015a

import pandas as pd
import numpy as np

# Import Data

In [2]:
dataset = BrainInvaders2015a()

def loadData(subject, session = 'session_1', run = 'run_1'):
    data = dataset._get_single_subject_data(subject)
    data = data[session][run]
    # data.set_montage(ten_twenty_montage)
    return data

data_subjects = []
subjects = list(range(1,44))
subjects.remove(1)
subjects.remove(27)

In [3]:
for subject in subjects:
    data_subjects.append(loadData(subject))

# Preprocessing

## Preprocessing Functions

In [4]:
import mne
from mne import create_info
from mne import Epochs, find_events

from sklearn.decomposition import PCA

def df_to_raw(df):
    sfreq = 512
    ch_names = list(df.columns)
    ch_types = ['eeg'] * (len(df.columns) - 1) + ['stim']
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

    df = df.T
      #mne looks at the tranpose() format
    df[:-1] *= 1e-6
      #convert from uVolts to Volts (mne assumes Volts data)

    info = create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)

    raw = mne.io.RawArray(df, info)
    raw.set_montage(ten_twenty_montage)
    return raw

def getEpochs(raw, event_id, tmin, tmax, picks):

    #epoching
    events = find_events(raw)
    
    #reject_criteria = dict(mag=4000e-15,     # 4000 fT
    #                       grad=4000e-13,    # 4000 fT/cm
    #                       eeg=100e-6,       # 150 μV
    #                       eog=250e-6)       # 250 μV

    reject_criteria = dict(eeg=100e-6)  #most voltage in this range is not brain components

    epochs = Epochs(raw, events=events, event_id=event_id, 
                    tmin=tmin, tmax=tmax, baseline=None, preload=True,verbose=False, picks=picks)  #8 channels
    print('sample drop %: ', (1 - len(epochs.events)/len(events)) * 100)

    return epochs
  
def preprocessing(rawdata, runPCA=False):
    # Convert and drop time column
    data_ses1_run1_pd = rawdata.to_data_frame()
    data_ses1_run1_pd = data_ses1_run1_pd.drop(['time'],axis = 1)
    raw = df_to_raw(data_ses1_run1_pd)

    # Notch Filter
    raw.notch_filter(np.arange(50, 251, 50))

    eeg_channels = mne.pick_types(raw.info, eeg=True)
    raw.filter(1,24,method = 'iir')


    if runPCA:
      raw_df = raw.to_data_frame()
      X1 = raw_df.drop(['time'],axis = 1)
      X = X1.drop(['STI 014'],axis = 1)
      y = raw_df['STI 014']
      pca = PCA(n_components=32)
      X = pca.fit(X.values).transform(X.values)
      y1 = y.values.reshape(-1,1)
      data = np.hstack((X,y1))
      df = pd.DataFrame(data, columns = list(X1.columns))
      raw = df_to_raw(df)

    event_id = {'NonTarget': 1, 'Target': 2}
    tmin = 0.0 #0
    tmax = 1.0 #0.5 seconds
    picks= eeg_channels
    epochs = getEpochs(raw,event_id, tmin, tmax, picks)

    X = epochs.get_data()
    y = epochs.events[:, -1]
    return X, y

## Preprocessing Data

In [5]:
from IPython.display import clear_output

X_subjects = []
y_subjects = []

for data in data_subjects:
    X, y = preprocessing(data)    
    X_subjects.append(X)
    y_subjects.append(y)
    clear_output(wait=True)

Creating RawArray with float64 data, n_channels=33, n_times=129472
    Range : 0 ... 129471 =      0.000 ...   252.873 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 24 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 24.00 Hz: -6.02, -6.02 dB

360 events found
Event IDs: [1 2]
sample drop %:  0.0


# Convert data

## Import torch

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset

## Data reshape, Convert to torch, Test/Train Split, and Data Loader

In [93]:
# Define dataset
def ShapePreparing(X, y, BATCH_SIZE = 32):
    X_reshaped = X[:, np.newaxis, :, :]
    torch_X_reshaped = torch.from_numpy(X_reshaped)
    torch_y = torch.from_numpy(y)

    ds = TensorDataset(torch_X_reshaped, torch_y)

    #Train test split
    train_size = int(round(torch_X_reshaped.size()[0] * 0.7))
    valid_size = int(round(torch_X_reshaped.size()[0] * 0.1))
    test_size = int(round(torch_X_reshaped.size()[0] * 0.2))
    sum_size = np.sum([train_size, valid_size, test_size])

    # Adjust total size to equal to sample size
    while sum_size<torch_X_reshaped.shape[0]:
        train_size += 1
        sum_size = np.sum([train_size, valid_size, test_size])
    while sum_size>torch_X_reshaped.shape[0]:
        train_size -= 1
        sum_size = np.sum([train_size, valid_size, test_size])
    
    # Split data
    train_set, valid_set, test_set = torch.utils.data.random_split(ds, [train_size, valid_size, test_size])

    #Train set loader
    train_iterator = torch.utils.data.DataLoader(dataset=train_set, 
                                            batch_size=BATCH_SIZE, 
                                            shuffle=True)
    #Validation set loader
    valid_iterator = torch.utils.data.DataLoader(dataset=valid_set, 
                                            batch_size=BATCH_SIZE, 
                                            shuffle=True)

    #Test set loader
    test_iterator = torch.utils.data.DataLoader(dataset=test_set, 
                                            batch_size=test_size, 
                                            shuffle=True)
    return train_iterator, valid_iterator, test_iterator

# Model

## CM-CW-CNN Model

In [122]:
class CM_CW_CNN(nn.Module):
    '''
    Expected Input Shape: (batch, channels, height , width)
    '''
    def __init__(self):
        super(CM_CW_CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1,16,kernel_size=(32,1),stride=(1,1)))
        self.conv2 = nn.Sequential(nn.Conv2d(16,16,kernel_size=(1,57),stride=(1,57)))
        self.fc = nn.Sequential(nn.Linear(144,144),nn.ReLU(),nn.Dropout(0.5),
                               nn.Linear(144,48),nn.ReLU(),nn.Dropout(0.5),
                               nn.Linear(48,12),nn.ReLU(),nn.Dropout(0.5),
                               nn.Linear(12,3),nn.ReLU(),nn.Dropout(0.1))
        # self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self,x):
        # print("X",x.shape)
        x = self.conv1(x)
        # print("X",x.shape)
        x = self.conv2(x)
        # print("X",x.shape)
        x = x.flatten(start_dim = 1)
        # print("X flatten",x.shape)
        # torch.manual_seed(9999)
        x = self.fc(x)
        #x = self.softmax(x)
        return x

In [123]:
import time

def train(model, iterator, optimizer, criterion, _print=False):
    total = 0
    correct = 0
    epoch_loss = 0
    epoch_acc = 0
    
    predicteds = []

    trues = []
    
    model.train()
    
    for batch, labels in iterator:
        
        #Move tensors to the configured device
        batch = batch.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(batch.float())
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        #check accuracy
        predictions = model(batch.float())
        _, predicted = torch.max(predictions.data, 1)  #returns max value, indices
        if _print:
            print('================== Predicted y ====================')
            print(predicted)
        predicteds.append(predicted)
        total += labels.size(0)  #keep track of total
        correct += (predicted == labels).sum().item()  #.item() give the raw number
        if _print:
            print('==================    True y   ====================')
            print(labels)
        trues.append(labels)
        acc = 100 * (correct / total)
                
        epoch_loss += loss.item()
        epoch_acc = acc
        
    return epoch_loss / len(iterator), epoch_acc, predicteds, trues

def evaluate(model, iterator, criterion):
    
    total = 0
    correct = 0
    epoch_loss = 0
    epoch_acc = 0
    
    predicteds = []
    trues = []
    
    model.eval()
    
    with torch.no_grad():
    
        for batch, labels in iterator:
            
            #Move tensors to the configured device
            batch = batch.to(device)
            labels = labels.to(device)

            predictions = model(batch.float())
            loss = criterion(predictions, labels)

            _, predicted = torch.max(predictions.data, 1)  #returns max value, indices
            predicteds.append(predicted)
            trues.append(labels)
            total += labels.size(0)  #keep track of total
            correct += (predicted == labels).sum().item()  #.item() give the raw number
            acc = 100 * (correct / total)
            
            epoch_loss += loss.item()
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator),predicteds, trues

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def saveModel(saveName, model, type='Best'):
    directory = 'results/'+(type(model).__name__)+'/'+type+'/'
    fileName = saveName+'.pth.tar'
    path = directory+fileName
    while True:
        try:
            torch.save(model.state_dict(), path)
            print("Model:{} saved.".format(fileName))
            break
        except:
            os.mkdir(directory)
            open(path, 'w')

## Training Function

In [124]:
from IPython.display import clear_output
import os


def Training(model, train_iterator, valid_iterator, N_EPOCHS = 50, saveName=None):
    
    best_valid_loss = float('inf')
    best_epoch = 0

    train_losses = []
    valid_losses = []

    train_accs = []
    valid_accs = []
    
    start_time_train = time.time()

    for epoch in range(N_EPOCHS):

        train_loss, train_acc, train_pred_label, train_true_label = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc, valid_pred_label, valid_true_label= evaluate(model, valid_iterator, criterion)
        train_losses.append(train_loss); train_accs.append(train_acc)
        valid_losses.append(valid_loss); valid_accs.append(valid_acc)
        
        if (epoch+1) % 5 == 0:
            clear_output(wait=True)
            print("Trainning:", saveName)
            print(f'Epoch: [{epoch+1:02}/{N_EPOCHS}]')
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}%')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc:.2f}%')
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_epoch = epoch
            if saveName != None:
                saveModel(saveName, model, type='Best')
    if saveName != None:
        saveModel(saveName, model, type='Last')
    training_time = time.time() - start_time_train
    return train_losses, valid_losses, train_accs, valid_accs, training_time, best_epoch



## Training Data

In [125]:
train_losses_list = []
train_accs_list = []
valid_losses_list = []
valid_accs_list = []
test_loss_list = []
test_acc_list = []
training_time_list = []
best_epoch_list = []
test_iterator_list = []

learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Configured device: ", device)

# for i in range(2):
for i in range(len(X_subjects)):
    # Split data
    train_iterator, valid_iterator, test_iterator = ShapePreparing(X_subjects[i], y_subjects[i], BATCH_SIZE=64)
    
    # Define model
    model = CM_CW_CNN()
    model = model.float()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model = model.to(device)
    criterion = criterion.to(device)
    
    # Training
    if subjects[i]<10:
        filename = "CM_CW_CNN_0"+str(subjects[i])
    else:
        filename = "CM_CW_CNN_"+str(subjects[i])
    # filename = None
    print(filename)
    train_losses, valid_losses, train_accs, valid_accs, training_time, best_epoch = Training(model, train_iterator, valid_iterator, N_EPOCHS = 50, saveName=filename)
    test_loss, test_acc, test_pred_label, test_true_label = evaluate(model, test_iterator, criterion)
    # Record results
    train_losses_list.append(train_losses); train_accs_list.append(train_accs)
    valid_losses_list.append(valid_losses); valid_accs_list.append(valid_accs)
    test_loss_list.append(test_loss); test_acc_list.append(test_acc)
    training_time_list.append(training_time); best_epoch_list.append(best_epoch)
    test_iterator_list.append(test_iterator)
    


Trainning: CM_CW_CNN_43
Epoch: [50/50]
	Train Loss: 0.248 | Train Acc: 84.13%
	 Val. Loss: 0.294 |  Val. Acc: 94.44%


In [ ]:
col = ["train_losses", "valid_losses", "test_loss", 
    "train_accs", "test_loss", "test_acc", 
    "training_time", "best_epoch"]
best_model_result = pd.DataFrame(columns=col, index=subjects)
last_model_result = pd.DataFrame(columns=col, index=subjects)

best_model_result.index.name = "subjects"

In [126]:
col = ["train_losses", "valid_losses", "test_loss", 
    "train_accs", "test_loss", "test_acc", 
    "training_time", "best_epoch"]
df = pd.DataFrame(columns=col, index=subjects)
df.index.name = "subjects"

for i in range(len(train_losses_list)):
    df.loc[subjects[i]] = [train_losses_list[i][-1], valid_losses_list[i][-1], test_loss_list[i], 
        train_accs_list[i][-1], valid_accs_list[i][-1], test_acc_list[i],
        training_time_list[i], best_epoch_list[i]]

df

,train_losses,valid_losses,test_loss,train_accs,test_loss,test_acc,training_time,best_epoch
subjects,,,,,,,,
2,0.271929,0.747743,0.935047,81.651376,87.234043,84.042553,3.471999,25
3,0.164678,0.412109,0.427783,89.285714,91.666667,91.666667,2.326001,37
4,0.247716,0.723783,0.652221,84.488449,83.72093,91.860465,3.270005,19
5,0.189037,2.250765,1.951431,88.778878,76.744186,81.395349,3.196,4
6,0.180359,0.662835,0.82696,89.68254,85.185185,87.037037,3.798,21
7,0.313407,0.506442,0.394013,68.650794,86.111111,88.888889,2.691001,28
8,0.177857,1.830348,1.367362,89.108911,76.744186,83.72093,3.143001,14
9,0.161374,0.645949,0.262867,89.153439,88.888889,93.518519,3.339,22
10,0.276874,2.382696,0.829944,85.903084,90.625,84.615385,1.934999,8


In [127]:
df.to_csv("results\CM_CW_CNN\CM_CW_CNN_result.csv")

In [128]:
df.mean()

train_losses      0.236050
valid_losses      1.270029
test_loss         0.922612
train_accs       84.370540
test_loss        83.936125
test_acc         85.783850
training_time     3.112122
best_epoch       19.317073
dtype: float64

In [138]:
test_loss_best = []
test_acc_best = []

for i, subject in enumerate(subjects):
    model = CM_CW_CNN()
    if subject<10:
        filename = "CM_CW_CNN_0"+str(subjects[i])
    else:
        filename = "CM_CW_CNN_"+str(subjects[i])
    path = 'results/CM_CW_CNN/'+filename+'.pth.tar'
    model.load_state_dict(torch.load(path))
    model.eval()
    model = model.float()

    criterion = nn.CrossEntropyLoss()

    model = model.to(device)
    criterion = criterion.to(device)

    test_loss, test_acc, test_pred_label, test_true_label = evaluate(model, test_iterator_list[1], criterion)
    test_loss_best.append(test_loss)
    test_acc_best.append(test_acc)


print("==== Best model ====")
# print(test_acc_best)
print("max: ", np.max(test_acc_best))
print("min: ", np.min(test_acc_best))
print("mean: ", np.mean(test_acc_best))

print("==== Last model ====")
# print(test_acc_list)
print("max: ", np.max(test_acc_list))
print("min: ", np.min(test_acc_list))
print("mean: ", np.mean(test_acc_list))


==== Best model ====
max:  91.66666666666666
min:  75.0
mean:  82.31707317073173
==== Last model ====
max:  95.08196721311475
min:  78.46153846153847
mean:  85.78385019901441


<class 'str'>


In [130]:
# if cuda.is_available():
#     net = model.cuda()
#     X_test_tensor = X_test_tensor.cuda()
# else:
#     net = net.cpu()
#     X_test_tensor = X_test_tensor.cpu()

# output = net(X_test_tensor)
# _, predicted = torch.max(output.data, 1)

## Results

In [131]:
# import matplotlib.pyplot as plt
# plt.plot(train_losses, label="train")
# plt.plot(valid_losses, label="validation")
# plt.title("Losses")
# plt.legend()
# plt.show()

In [132]:
# plt.plot(train_accs, label="train")
# plt.plot(valid_accs, label="validation")
# plt.title("Accuracy")
# plt.legend()
# plt.show()